<a href="https://colab.research.google.com/github/alicelindel3/ibm5100/blob/main/cp3/Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

最終課題で具体的にどのような要件が求められているかについては、最終課題RAEDMEをご覧ください。ここでは最終課題の取り組み方のサンプルとして、第二回のコンペのHome credit社のデータを利用してHome credit社での事業提案例を考えてみます。
**実際の最終課題では、このHome Credit社のデータを用いているとは限らないので、最終課題で指定されたデータセットを解析してください。**
データ分析や機械学習を行うことで、その企業の課題やニーズを見出し、問題解決に役立てる事業を提案してみましょう。

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train.csv')
#df_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/test.csv')
#今回は最終課題を想定してdf_testは使わないことにします。
#pathは各自で変更してください。

最初にデータサイズや型、訓練データの最初の10個のデータを確認します。

In [ ]:
print('データのデータ数は{}、変数は{}種類です。'.format(df.shape[0], df.shape[1]))

In [ ]:
print(df.info())

In [ ]:
df.head(10)

欠損値の多いカラムが見られることが分かったので、欠損値の数をカウントします。

In [ ]:
df.isnull().sum() 

多くのカラムで欠損値があることが分かりました。これだけ欠損値が多いと、単純にカラムを削除するなどの方法ではうまくいかない可能性も考えられます。次に、事業の例として考えやすそうなTARGET(債務不履行の有無)がどのようなデータなのか確認します。

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
df['TARGET'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('TARGET')
ax[0].set_ylabel('')
sns.countplot('TARGET',data=df,ax=ax[1])
ax[1].set_title('TARGET')
plt.show()

TARGETは不均衡データであることが分かりました。全体についてEDAを行ったり、TARGETについて関連を調べるのも有効ですが(詳しくは、第二回コンペのチューリアルをご覧ください。)、今回は非常にデータが多いので、まずLightGBMでどのような特徴量がTARGETに関連しているのか調べてみることにします。LightGBMは欠損値があっても学習することができるので、今回のデータ分析における機械学習では有効な手段の一つと予測されます。

In [ ]:
df.columns[df.dtypes.values == "object"]

In [ ]:
categ_nominal = ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'ORGANIZATION_TYPE']

In [ ]:
for i in categ_nominal:
    df = pd.concat([df, pd.get_dummies(df[i], prefix=i, dummy_na=True)], sort=False, axis=1)
df = df.drop(categ_nominal, axis=1)
df.info()

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [ ]:
X = df.drop('TARGET',axis=1) 
y = df['TARGET'] # 目的変数
# トレーニングデータ,テストデータの分割
X_train, X_valid, y_train, y_valid = train_test_split(X, y,test_size=0.2, random_state=0)

今回は不均衡データであることから、評価指標にはAUCを用いることにします。適切な評価指標を選ぶことは最終課題でも重要です。

In [ ]:
# 学習に使用するデータを設定
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train) 

# パラメータ
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary', # 目的 : 分類
        'metric': {'auc'},
         'num_leaves': 20,             
        'max_depth':5,
        'min_data_in_leaf': 3,        
        'num_iteration': 100,            
        'learning_rate':0.03,
        'num_boost_round':100,
         'early_stopping_rounds':20, 
}

# モデルの学習
model = lgb.train(params,
                  train_set=lgb_train, # トレーニングデータの指定
                  valid_sets=lgb_eval, # 検証データの指定
                  )

# テストデータの予測
y_pred = model.predict(X_valid)

モデルの学習が終わりました。決定木ベースのアンサンブル分析器では特徴量の重要度を算出することができるため、可視化してみます。

In [ ]:
lgb.plot_importance(model, height=0.5, figsize=(30,40))

モデルの学習に当たって重要と判断された特徴量について可視化することができました。重要と判断された特徴量に関していくつか検討してみることにします。それぞれの特徴量に関して、平均値や中央値、データの分布を確認し、債務不履行の有無でウィルコクソンの順位和検定を有意水準を5%として検定し、2群の中央値に有意差があるか検討してみます。

まずはEXT_SOURCE_1に関して検討します。

In [ ]:
df.groupby(["TARGET"])["EXT_SOURCE_1"].mean()

In [ ]:
df.groupby(["TARGET"])["EXT_SOURCE_1"].median()

実際に債務不履行の有無によるEXT_SOURCE_1の中央値やヒストグラムを可視化してみましょう。

In [ ]:
sns.barplot(x='TARGET', y='EXT_SOURCE_1', data=df, estimator=np.median)

In [ ]:
df.groupby('TARGET')['EXT_SOURCE_1'].plot.hist(bins=20, alpha=0.5, legend=True)

ウィルコクソンの順位和検定を有意水準を5%として検定し、2群の中央値に有意差があるか検討してみます。

In [ ]:
from scipy import stats
s, pvalue = stats.mannwhitneyu(df[df["TARGET"]==1]["EXT_SOURCE_1"]
                , df[df["TARGET"]==0]["EXT_SOURCE_1"]
                ,alternative='two-sided')
pvalue < 0.05

この検定より、債務不履行の有無に関してEXT_SOURCE_1には有意差があり、カラム名から察するに外部からの、この信用スコアのようなものが高いと債務不履行になりにくいことが分かりました。反対に、顧客データから顧客の信用スコアを求めることで他社に有益な情報を供与できるかもしれません。信用スコアを他社に提供する分、顧客に還元するアイデアなども考えられます。


次に、DAYS_BIRTHについて検討します。

In [ ]:
df.groupby(["TARGET"])["DAYS_BIRTH"].mean()

In [ ]:
df.groupby(["TARGET"])["DAYS_BIRTH"].median()

In [ ]:
sns.barplot(x='TARGET', y='DAYS_BIRTH', data=df, estimator=np.median)

In [ ]:
df.groupby(["TARGET"])["DAYS_BIRTH"].plot.hist(bins=20, alpha=0.5, legend=True)

In [ ]:
s, pvalue = stats.mannwhitneyu(df[df["TARGET"]==1]["DAYS_BIRTH"]
                , df[df["TARGET"]==0]["DAYS_BIRTH"]
                ,alternative='two-sided')
pvalue < 0.05

この検定より、債務不履行の有無に関してDAYS_BIRTHには有意差があることが分かりました。値を鑑みるに、年齢は数字の絶対値が小さい方が若いと考えられるので、若い人の方が債務不履行になりやすいと言え、若年層が返済しやすい融資プランの需要があることが予想できます。

次に、AMT_CREDIT(融資額)について検討します。

In [ ]:
df.groupby(["TARGET"])["AMT_CREDIT"].mean()

In [ ]:
df.groupby(["TARGET"])["AMT_CREDIT"].median()

In [ ]:
sns.barplot(x='TARGET', y='AMT_CREDIT', data=df, estimator=np.median)

In [ ]:
df.groupby(["TARGET"])["AMT_CREDIT"].plot.hist(bins=20, alpha=0.5, legend=True)

In [ ]:
s, pvalue = stats.mannwhitneyu(df[df["TARGET"]==1]["AMT_CREDIT"]
                , df[df["TARGET"]==0]["AMT_CREDIT"]
                ,alternative='two-sided')
pvalue < 0.05

この検定より、債務不履行の有無に関してAMT_CREDITには有意差があることが分かりました。値が小さい方が債務不履行になりやすいという結果なので、問題は借入金の総額ではなく、収入との問題があるのかもしれません。収入AMT_INCOME_TOTALに関して検討してみます。

In [ ]:
df.groupby(["TARGET"])["AMT_INCOME_TOTAL"].mean()

In [ ]:
df.groupby(["TARGET"])["AMT_INCOME_TOTAL"].median()

In [ ]:
sns.barplot(x='TARGET', y='AMT_INCOME_TOTAL', data=df, estimator=np.median)

In [ ]:
df.groupby(["TARGET"])["AMT_INCOME_TOTAL"].plot.hist(bins=20, alpha=0.5, legend=True)

In [ ]:
df.groupby(["TARGET"])["AMT_INCOME_TOTAL"].plot.hist(bins=20, alpha=0.5,log=True, legend=True)
#スケールが見にくいので対数変換

In [ ]:
s, pvalue = stats.mannwhitneyu(df[df["TARGET"]==1]["AMT_INCOME_TOTAL"]
                , df[df["TARGET"]==0]["AMT_INCOME_TOTAL"]
                ,alternative='two-sided')
pvalue < 0.05

この検定より、債務不履行の有無に関してAMT_INCOME_TOTALには有意差があることが分かりました。値が小さい方が債務不履行になりやすいという結果なので、収入が少ない人の方が債務不履行になりやすいということが分かりました。ここから、収入が少ない人への適切な融資プランの需要が考えられます。

先ほど予想したように、借入金が多ければ債務不履行になりやすいという訳ではないようなので、収入と借入金の割合について考えてみたいと思います。

In [ ]:
df['Ratio_of_income_to_debt']=df['AMT_CREDIT']/df['AMT_INCOME_TOTAL']
df.groupby(["TARGET"])["Ratio_of_income_to_debt"].mean()

In [ ]:
df.groupby(["TARGET"])["Ratio_of_income_to_debt"].median()

In [ ]:
sns.barplot(x='TARGET', y='Ratio_of_income_to_debt', data=df, estimator=np.median)

In [ ]:
df.groupby('TARGET')['Ratio_of_income_to_debt'].plot.hist(bins=20, alpha=0.5, legend=True)

In [ ]:
s, pvalue = stats.mannwhitneyu(df[df["TARGET"]==1]["Ratio_of_income_to_debt"]
                , df[df["TARGET"]==0]["Ratio_of_income_to_debt"]
                ,alternative='two-sided')
pvalue < 0.05

どうやら、債務不履行かどうかは年収と借入金の比では決定しないようです。収入が少ない人の場合、借入金の割合や額が少なくても債務不履行になるケースが多いのかもしれません。

今回は上手くいきませんでしたが、仮説を立ててデータを色々な角度から分析するのは企業の課題を見つけるのに有効な手段の一つです。

ここまでのデータ分析から、**外部の信用スコアが重要であることや、若者や収入の少ない人は債務不履行者になりやすい課題**が見えてきました。
今回は簡便な方法でEDAを行いましたが、更に深く探索を行うことや、全く別の観点からEDAを行うことは評価に繋がります。

今回はEDAの手段としてLightGBMを用いましたが、この学習結果をモデル構築の一部とすることも可能です。**最終課題では、データ分析結果を可視化して提示し、どのようなことが明らかになったのか示すとともに、機械学習によるモデル構築を行いましょう。この流れに沿って行けば、どのような事業提案を行ったらよいかが見えてくると思います。**

tips:可視化を行う際に、日本語を軸の名前などに記述したい時にはjapanize_matplotlibを用いると便利です。調べてみてください。

最終課題の説得力を上げる一つの方法としては、事業に定量性を持たせることです。機械学習モデルを用いて、例えば今回のケースなら債務不履行者による損失がどれだけ減らせるか示すことができれば、定量性が上がり説得力が増します。


まず、機械学習する前のHome Credit社の損失合計を求めてみます。債務不履行者の合計金額を求めればいいですから、

In [ ]:
loss_sum=df[df["TARGET"]==1]
loss_sum['AMT_CREDIT'].sum()

これより、通貨単位をドルとして、7665723153ドルの損失があることが分かりました。ここで、今回機械学習で債務不履行者と予測した融資金額の合計を算出します。なお、債務不履行者かどうかは、実際の債務不履行者の割合が8%程度であることを考え、今回は大雑把ですが予測確率が上位の5%より小さいかどうかで判定してみます。

In [ ]:
X_valid['pred']=y_pred
X_valid['pred'].hist()

In [ ]:
threshold=X_valid['pred'].quantile(0.95)
print(threshold)
X_valid.loc[X_valid['pred'] < threshold, 'pred'] = 0
X_valid.loc[X_valid['pred'] >= threshold, 'pred'] = 1

In [ ]:
loss_pred=X_valid[X_valid["pred"]==1]
loss_pred['AMT_CREDIT'].sum()

In [ ]:
787740439.5/0.8

債務不履行者と予測した人への融資金額は787740439.5ドルでした。訓練データ・テストデータの分割(8割と2割)を考慮すると、これらがほぼランダムに分割できているのであれば984675549.375ドルの損失があると言えます。機械学習を用いて債務不履行者と判定した人に融資を行わなければ、これだけの損失を回避することができます。ただし、この計算ではモデルの性能等を全く考慮していないので、やや乱暴な理論と言えます。しかし、ただ機械学習をして債務不履行者を予測します、と事業提案されるよりは具体的で分かりやすい説明になると言えます。

本サンプルでは行っておりませんが、**市場調査を行うことも説得力の観点で大事**です。例えば同業他社ではどのような問題を抱えており、その課題をいかにデータで解決できるか示すことができれば説得力を増すことができます。

以上で最終課題のサンプルとします。

最終課題の評価を上げるには、データ分析を綿密に行うこと、モデルの性能を高めること、斬新な事業を提案すること、隙のない議論に基づいた定量性のある事業を提案することなどが考えられます。また、事業提案ですので、ビジネスモデルキャンバスやリーンキャンバスを用いることも有効だと考えられます。ぜひ調べてみてください。非常にチャレンジングな課題ですが、今まで学んだことを活かしてデータ分析や機械学習を用いた事業を提案して下さい。皆さんの力作をお待ちしております。